# 크롤러

In [1]:
import pandas as pd
import requests

def get_bok_df(info, classification,start_data, end_data, specific_info):
    apikey = "HF0ML1Y9A0RN7R7K4JE2"
    url = 'https://ecos.bok.or.kr/api/StatisticSearch/' + apikey + '/json/kr/1/100/817Y002/D/20180101/20221231/010200000'
    response = requests.get(url)

    result = response.json()
    list_total_count=(int)(result['StatisticSearch']['list_total_count'])
    list_count=(int)(list_total_count/100) + 1
    rows=[]
    for i in range(0,list_count):
        start = str(i * 100 + 1)
        end = str((i + 1) * 100)
        url = 'https://ecos.bok.or.kr/api/StatisticSearch/' + apikey + '/json/kr/' \
                + start + '/' + end + f"/{info}/{classification}/{start_data}/{end_data}/{specific_info}" 
        response = requests.get(url)
        result = response.json()
        rows = rows + result['StatisticSearch']['row']
    df=pd.DataFrame(rows)
    return df


## 파라미터 정보

In [11]:
# 원화의 대미 달러 종가기준 연평균
param_uskor_exchange = ['731Y006', 'A', 2018, 2022, '0000003']

# M2 경제주체별 연평균, 단위 십억원
param_m2 = ['101Y015', 'A', 2018, 2022, 'BBHA00']

# 생산자물가지수 연평균, 2015년 PPI를 100으로 함.
param_PPI = ['404Y014', 'A', 2018, 2022, '*AA']


# 한국 정책금리 월단위
param_kr_policyratio = ['902Y006', 'M', 201801, 202212, 'KR']
# 미국 정책금리 월단위
param_us_policyratio = ['902Y006', 'M', 201801, 202212, 'US']

# # 한국은행 기준금리 일단위
# param_kor_standard_yield = ['722Y001', 'D', 20180101, 20221231, '0101000']
# # 국고채 3년물 금리 일단위 -> 집계필요
# param_kor_govrn_3ybond = ['817Y002', 'D', 20180101, 20221231, '010200000'] 

In [20]:
# 환율, 통화량, 생산자 물가지수 연평균
uskor_exchange_year = get_bok_df(*param_uskor_exchange)
m2_year = get_bok_df(*param_m2)
PPI_year = get_bok_df(*param_PPI)

# 월단위 정책금리
kr_policyratio_month  = get_bok_df(*param_kr_policyratio)
us_policyratio_month = get_bok_df(*param_us_policyratio)

### 최저임금 데이터 로드 - csv file

In [22]:
minimum_wage_1423 = pd.read_csv("C:/Users/USER_20211027/Desktop/final_project/github/Final-Project/data/고용노동부_연도별 최저임금_2014_2023.csv", encoding="utf-8")

## 데이터 프레임으로 저장

# DB 전송

In [25]:
def pythondf_to_DB(dataframe_name, database_name, desired_table_name):
	import pymysql
	from sqlalchemy import create_engine
	from sqlalchemy.orm import sessionmaker
	username = 'multi'
	password = 'Campus123!'
	hostname =  'ec2-15-152-211-160.ap-northeast-3.compute.amazonaws.com'
	database_name = database_name
	desired_table_name = desired_table_name
	cnx = pymysql.connect(user=username, password=password, host=hostname)
	cursor = cnx.cursor()
	engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
						.format(user=username,
								pw=password,
								db=database_name,
								host=hostname))
	Session = sessionmaker(bind=engine)
	session = Session()
	try:
		# Change df name
		dataframe_name.to_sql(desired_table_name, con=engine, if_exists='replace', index=False, chunksize = 1000)
		session.commit()
	except:
		session.rollback()
		raise
	finally:
		session.close()
	cursor.close()

## Data Lake upload

In [27]:
# def pythondf_to_DB(dataframe_name, "database_name", "desired_table_name")

# 연평균 원/달러 환율
pythondf_to_DB(uskor_exchange_year, "Data_Lake", "uskor_exchange_year")

# 연평균 통화량
pythondf_to_DB(m2_year, "Data_Lake", "m2_year")

# 연평균 생산자 물가
pythondf_to_DB(PPI_year, "Data_Lake", "PPI_year")

# 한국, 미국 월단위 정책금리
pythondf_to_DB(kr_policyratio_month, "Data_Lake", "kr_policyratio_month")
pythondf_to_DB(us_policyratio_month, "Data_Lake", "us_policyratio_month")

# 연별 최저임금
pythondf_to_DB(minimum_wage_1423, "Data_Lake", "minimum_wage_1423")

In [30]:
kr_policyratio_month['TIME'] = pd.to_datetime(kr_policyratio_month['TIME'], format='%Y%m')
kr_policyratio_month['YEAR'] = kr_policyratio_month['TIME'].dt.year
kr_policyratio_month['DATA_VALUE'] = kr_policyratio_month['DATA_VALUE'].astype(float)
kr_policyratio_year = kr_policyratio_month.groupby('YEAR')['DATA_VALUE'].mean().round(4)

us_policyratio_month['TIME'] = pd.to_datetime(us_policyratio_month['TIME'], format='%Y%m')
us_policyratio_month['YEAR'] = us_policyratio_month['TIME'].dt.year
us_policyratio_month['DATA_VALUE'] = us_policyratio_month['DATA_VALUE'].astype(float)
us_policyratio_year = us_policyratio_month.groupby('YEAR')['DATA_VALUE'].mean().round(4)

# Data Warehouse

In [35]:
# 한국정책금리 - 미국정책금리 -> 양수값이 나오면 한국정책금리 > 미국정책금리
kr_minus_us_policyratio = kr_policyratio_year - us_policyratio_year
krus_diff_policyratio = pd.DataFrame({'year': kr_minus_us_policyratio.index, 'kr_minus_us_rate': kr_minus_us_policyratio.values})

# # 결과 출력
# print(df_kr_minus_us_policyratio)

In [36]:
krus_diff_policyratio

,year,kr_minus_us_rate
0,2018,-0.2916
1,2019,-0.5625
2,2020,0.2917
3,2021,0.5208
4,2022,0.2292


In [44]:
uskor_exchange_year_average = uskor_exchange_year[uskor_exchange_year.ITEM_NAME2=="평균자료"]

In [46]:
df = uskor_exchange_year_average[['TIME', 'DATA_VALUE']]

In [47]:
df

,TIME,DATA_VALUE
0,2018,1100.58
1,2019,1166.11
2,2020,1180.01
3,2021,1144.61
4,2022,1292.2


## 파라미터 정보

In [ ]:
# 원화의 대미 달러 종가기준 연평균
param_uskor_exchange = ['731Y006', 'A', 2018, 2022, '0000003']

# M2 경제주체별 연평균, 단위 십억원
param_m2 = ['101Y015', 'A', 2018, 2022, 'BBHA00']

# 생산자물가지수 연평균, 2015년 PPI를 100으로 함.
param_PPI = ['404Y014', 'A', 2018, 2022, '*AA']


# 한국 정책금리 월단위
param_kr_policyratio = ['902Y006', 'M', 201801, 202212, 'KR']
# 미국 정책금리 월단위
param_us_policyratio = ['902Y006', 'M', 201801, 202212, 'US']

# # 한국은행 기준금리 일단위
# param_kor_standard_yield = ['722Y001', 'D', 20180101, 20221231, '0101000']
# # 국고채 3년물 금리 일단위 -> 집계필요
# param_kor_govrn_3ybond = ['817Y002', 'D', 20180101, 20221231, '010200000'] 

In [ ]:
# 환율, 통화량, 생산자 물가지수 연평균
df_uskor_exchange = get_bok_df(*param_uskor_exchange)
df_m2 = get_bok_df(*param_m2)
df_PPI = get_bok_df(*param_PPI)

# 월단위 정책금리
df_kr_policyratio  = get_bok_df(*param_kr_policyratio)
df_us_policyratio = get_bok_df(*param_us_policyratio)

### 최저임금 데이터 로드 - csv file

In [ ]:
df_minimum_wage = pd.read_csv("C:/Users/USER_20211027/Desktop/final_project/github/Final-Project/data/고용노동부_연도별 최저임금_2014_2023.csv")

# DB 전송

In [ ]:
def pythondf_to_DB(dataframe, database_name, table_name):
	import pymysql
	from sqlalchemy import create_engine
	from sqlalchemy.orm import sessionmaker
	username = 'multi'
	password = 'Campus123!'
	hostname =  'ec2-15-152-211-160.ap-northeast-3.compute.amazonaws.com'
	database_name = database_name
	desired_table_name = table_name
	cnx = pymysql.connect(user=username, password=password, host=hostname)
	cursor = cnx.cursor()
	engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
						.format(user=username,
								pw=password,
								db=database_name,
								host=hostname))
	Session = sessionmaker(bind=engine)
	session = Session()
	try:
		# Change df name
		dataframe.to_sql(desired_table_name, con=engine, if_exists='replace', index=False, chunksize = 1000)
		session.commit()
	except:
		session.rollback()
		raise
	finally:
		session.close()
	cursor.close()

## Data Lake upload

In [ ]:
# def pythondf_to_DB(dataframe, database_name, table_name):

# 미국한국 환율
pythondf_to_DB(df_uskor_exchange, database_name, df_uskor_exchange)

# 연평균 통화량
pythondf_to_DB(df_m2, database_name, df_m2)

# 연평균 생산자 물가
pythondf_to_DB(df_PPI, database_name, df_PPI)

# 한국 미국 월단위 정책금리
pythondf_to_DB(df_kr_policyratio, database_name, df_kr_policyratio)
pythondf_to_DB(df_us_policyratio, database_name, df_us_policyratio)

# 최저임금
pythondf_to_DB(df_minimum_wage, database_name, df_minimum_wage)


NameError: name 'pythondf_to_DB' is not defined

In [ ]:
df_kr_policyratio['TIME'] = pd.to_datetime(df_kr_policyratio['TIME'], format='%Y%m')
df_kr_policyratio['YEAR'] = df_kr_policyratio['TIME'].dt.year
df_kr_policyratio['DATA_VALUE'] = df_kr_policyratio['DATA_VALUE'].astype(float)
kr_year_policyratio = df_kr_policyratio.groupby('YEAR')['DATA_VALUE'].mean().round(4)

df_us_policyratio['TIME'] = pd.to_datetime(df_us_policyratio['TIME'], format='%Y%m')
df_us_policyratio['YEAR'] = df_us_policyratio['TIME'].dt.year
df_us_policyratio['DATA_VALUE'] = df_us_policyratio['DATA_VALUE'].astype(float)
us_year_policyratio = df_us_policyratio.groupby('YEAR')['DATA_VALUE'].mean().round(4)

In [ ]:
# 한국정책금리 - 미국정책금리 -> 양수값이 나오면 한국정책금리 > 미국정책금리
kr_minus_us_policyratio = kr_year_policyratio - us_year_policyratio
df_kr_minus_us_policyratio = pd.DataFrame({'YEAR': kr_minus_us_policyratio.index, 'KR MINUS US RATE': kr_minus_us_policyratio.values})

# # 결과 출력
# print(df_kr_minus_us_policyratio)

In [ ]:
df_minimum_wage = pd.read_csv("고용노동부_연도별 최저임금_20220805.csv")

In [ ]:
kr_minus_us_policyratio

YEAR
2018   -0.2916
2019   -0.5625
2020    0.2917
2021    0.5208
2022    0.2292
Name: DATA_VALUE, dtype: float64

In [ ]:
kr_minus_us_policyratio

YEAR
2018   -0.2916
2019   -0.5625
2020    0.2917
2021    0.5208
2022    0.2292
Name: DATA_VALUE, dtype: float64